In [74]:
import conformance_functions as cf
import discovery_functions as disc_f
import xmltodict
import pandas as pd
import uuid
import plotly.express as px

In [2]:
with open('./data/discovered_process.bpmn', 'r', encoding='utf-8') as file:
    discovered_xml = file.read()
with open('./data/diagram_1.bpmn', 'r', encoding='utf-8') as file:
    normative_xml = file.read()

discovered_bpmn = xmltodict.parse(discovered_xml)
normative_bpmn = xmltodict.parse(normative_xml)

In [3]:
discovered_tilt_bpmn_elements = cf.filter_list_for_element(
    cf.get_list_of_elements_containing_key(discovered_bpmn,"bpmn:extensionElements",[]),
    key="bpmn:extensionElements",
    nested_key="tilt:dataDisclosed"
    )
normative_tilt_bpmn_elements = cf.filter_list_for_element(
    cf.get_list_of_elements_containing_key(normative_bpmn,"bpmn:extensionElements",[]),
    key="bpmn:extensionElements",
    nested_key="tilt:dataDisclosed"
    )

In [4]:
discovered_data_disclosed_elements = \
    cf.extract_data_disclosed_from_element_list(discovered_tilt_bpmn_elements)
normative_data_disclosed_elements = \
    cf.extract_data_disclosed_from_element_list(normative_tilt_bpmn_elements)

In [5]:
discovered_new_ids = cf.get_data_disclosed_dicts(discovered_tilt_bpmn_elements)
normative_new_ids = cf.get_data_disclosed_dicts(normative_tilt_bpmn_elements)

In [6]:
df_discovered__new = pd.DataFrame(discovered_new_ids,columns=["id_disc","dict","old_id"]).dropna()
df_discovered_old = pd.DataFrame(discovered_data_disclosed_elements,columns=["name","old_id"]).dropna()
df_data_disclosed_discovered = pd.merge(df_discovered__new,df_discovered_old,how="outer",on="old_id").drop_duplicates(["id_disc","name","old_id"])

In [7]:
df_normative_new = pd.DataFrame(normative_new_ids,columns=["id_norm","dict","old_id"]).dropna()
df_normative_old = pd.DataFrame(normative_data_disclosed_elements,columns=["name","old_id"]).dropna()
df_data_disclosed_normative = pd.merge(df_normative_new,df_normative_old,how="outer",on="old_id").drop_duplicates(["id_norm","name","old_id"])

In [8]:
combined_df = pd.concat([df_data_disclosed_discovered.rename(columns={"id_disc":"id"}),df_data_disclosed_normative.rename(columns={"id_norm":"id"})])
combined_df["category"] = combined_df.apply(lambda x: x["dict"]["@category"],axis=1)
df_purpsoes = cf.build_tilt_element_df(combined_df,"tilt:purposes")
df_legal_bases = cf.build_tilt_element_df(combined_df,"tilt:legalBases")

In [9]:
df_disclosed_delta = pd.merge(df_data_disclosed_discovered[["id_disc","name"]],df_data_disclosed_normative[["id_norm","name"]],how="outer",left_on=["id_disc","name"],right_on=["id_norm","name"])#.drop_duplicates()

In [10]:
df_el = pd.read_csv("./data/transparency_event_log.csv",sep=",")
df_el["time:timestamp"] = pd.to_datetime(df_el["time:timestamp"],format="%Y-%m-%d %H:%M:%S,%f")
df_log_dd = disc_f.get_data_disclosed_df(df_el)
df_log_dd = pd.merge(df_el,df_log_dd[["ident:eid","id"]].rename(columns={"id":"tilt:dataDisclosed:id"}),how="outer",on="ident:eid")#.drop(columns=["tilt:data_disclosed","tilt:purposes","tilt:legal_bases","message"])

/home/ciphersmaug/Projects/master/transparency-logging/src/mining-dashboard/app/discovery_functions.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd_df[c] = dd_df[c].apply(lambda x: str(x))
/home/ciphersmaug/Projects/master/transparency-logging/src/mining-dashboard/app/discovery_functions.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_dd["tilt:element"] = unique_dd.apply(build_data_disclosed_element,axis=1)


In [11]:
df_disclosed_delta[~pd.isna(df_disclosed_delta["id_norm"])]

,id_disc,name,id_norm
1,19748727-0566-5a01-bf0a-43c4260f39cd,Process User Request,19748727-0566-5a01-bf0a-43c4260f39cd
14,7a0bd858-6c68-517d-88a2-0d3156d69d4e,Gather User Address,7a0bd858-6c68-517d-88a2-0d3156d69d4e
16,0101ffee-a1e8-599e-89ed-75f8574d50e3,Gather User Address,0101ffee-a1e8-599e-89ed-75f8574d50e3
18,ae78878f-caf4-5cac-8c8c-8f7730b0f6dc,Gather User Address,ae78878f-caf4-5cac-8c8c-8f7730b0f6dc
24,NaN,Inform Newsletter Service,fb924372-41c3-522b-8e53-11805cfd395c


In [12]:
#df_el[df_el["tilt:dataDisclosed:id"].isin(df_disclosed_delta[])]

In [13]:
df_disclosed_delta[["name","id_norm"]].rename(columns={"name":"concept:name","id_norm":"tilt:dataDisclosed:id"}).to_dict(orient="list")

{'concept:name': ['Send Data to User',
  'Process User Request',
  'Request User Address',
  'Send Newsletter',
  'Inform Newsletter Service',
  'Send Newsletter',
  'Inform Newsletter Service',
  'Send Newsletter',
  'Inform Newsletter Service',
  'Send Newsletter',
  'Inform Newsletter Service',
  'Send Newsletter',
  'Inform Newsletter Service',
  'Inform Newsletter Service',
  'Gather User Address',
  'Combine User Data',
  'Gather User Address',
  'Combine User Data',
  'Gather User Address',
  'Combine User Data',
  'Read Last Login',
  'Combine User Data',
  'Combine User Data',
  'Combine User Data',
  'Inform Newsletter Service'],
 'tilt:dataDisclosed:id': [nan,
  UUID('19748727-0566-5a01-bf0a-43c4260f39cd'),
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  UUID('7a0bd858-6c68-517d-88a2-0d3156d69d4e'),
  nan,
  UUID('0101ffee-a1e8-599e-89ed-75f8574d50e3'),
  nan,
  UUID('ae78878f-caf4-5cac-8c8c-8f7730b0f6dc'),
  nan,
  nan,
  nan,
  nan,
  

In [14]:
df_disclosed_delta.iloc[0]["id_disc"]

UUID('19748727-0566-5a01-bf0a-43c4260f39cd')

In [15]:
def isCompliant(row:pd.Series,df_disclosed_delta)->pd.Series:
    id = str(row['tilt:dataDisclosed:id'])
    df_disclosed_delta["id_norm"] = df_disclosed_delta["id_norm"].apply(lambda x: str(x))
    result = df_disclosed_delta.query(f"id_norm == '{id}' and name == '{row['concept:name']}'")#[df_disclosed_delta["id_norm"] == row["tilt:dataDisclosed:id"]] and df_disclosed_delta["name"]==row["concept:name"]]
    #result = df_disclosed_delta.loc[(df_disclosed_delta["id_norm"]==id)and(df_disclosed_delta["name"]==row["concept:name"])]
    if len(result) <1:
        return False
    else:
        return True


In [16]:
str(uuid.UUID("19748727-0566-5a01-bf0a-43c4260f39cd"))

'19748727-0566-5a01-bf0a-43c4260f39cd'

In [17]:
df_log_dd["tilt:isCompliant"] = df_log_dd.apply(lambda x: isCompliant(x,df_disclosed_delta),axis=1)

In [18]:
#df_log_dd["test:compliant"].value_counts()

In [19]:
#compliance = df_log_dd[["tilt:dataDisclosed:id","concept:name"]].isin(df_disclosed_delta[["name","id_norm"]].rename(columns={"name":"concept:name","id_norm":"tilt:dataDisclosed:id"}))

In [20]:
#compliance["concept:name"].value_counts()

In [21]:
#df_log_dd["tilt:isCompliant"] = compliance.apply(lambda x: x[0]==x[1]==True,axis=1)

In [22]:
def calcualte_percentage_of_compliance(g,column = "tilt:isCompliant",new_colum = "tilt:percentageCompliant"):
    g[new_colum] = g[column].sum()/len(g[column])
    return g

In [23]:
#for i,g in df_log_dd.groupby(["case:concept:name","time:timestamp","concept:name"]):
#    if calcualte_percentage_of_compliance(g)["tilt:percentageCompliant"].min() != 1.0 and len(g)>1:
#        print(g[["message","tilt:dataDisclosed:id"]])
#        break

In [24]:
df_activity_compliance = df_log_dd.groupby(["case:concept:name","time:timestamp","concept:name"]).apply(lambda x : calcualte_percentage_of_compliance(x))#.drop_duplicates(["case:concept:name","time:timestamp","concept:name"])

/tmp/ipykernel_1490061/3620264725.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_activity_compliance = df_log_dd.groupby(["case:concept:name","time:timestamp","concept:name"]).apply(lambda x : calcualte_percentage_of_compliance(x))#.drop_duplicates(["case:concept:name","time:timestamp","concept:name"])


In [25]:
df_cases_compliant = df_activity_compliance.groupby(["case:concept:name"]).apply(lambda x : calcualte_percentage_of_compliance(x,"tilt:percentageCompliant","tilt:case:percentageCompliant")).drop_duplicates(["case:concept:name"])

/tmp/ipykernel_1490061/1579976359.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_cases_compliant = df_activity_compliance.groupby(["case:concept:name"]).apply(lambda x : calcualte_percentage_of_compliance(x,"tilt:percentageCompliant","tilt:case:percentageCompliant")).drop_duplicates(["case:concept:name"])


In [26]:
df_cases_compliant["tilt:case:percentageCompliant"].value_counts().sort_index()

0.111111       2
0.166667     291
0.235294       3
0.307692    1217
0.333333      20
0.500000      67
0.666667      26
1.000000      36
Name: tilt:case:percentageCompliant, dtype: int64

In [55]:
incompliant_activities_df = pd.DataFrame(df_activity_compliance[~df_activity_compliance["tilt:isCompliant"]]["tilt:dataDisclosed:id"].value_counts().sort_values(ascending=False)).reset_index().rename(columns={"index":"tilt:dataDisclosed:id","tilt:dataDisclosed:id":"count"})

In [86]:
bar_data = df_cases_compliant["tilt:case:percentageCompliant"].value_counts().apply(lambda x: x/df_cases_compliant["tilt:case:percentageCompliant"].value_counts().sum()).sort_index(ascending=True)
bar_data.index.name = "Percent Compliant"
bar_data = bar_data.rename(index="Percent of all Cases")
bar_data = bar_data * 100
fig = px.bar(bar_data,title="Case percentage of data disclosed compliance")
fig.show()

In [137]:
bar_data.reset_index()

,Percent Compliant,Percent of all Cases
0,0.111111,0.120337
1,0.166667,17.509025
2,0.235294,0.180505
3,0.307692,73.225030
4,0.333333,1.203369
5,0.500000,4.031288
6,0.666667,1.564380
7,1.000000,2.166065


In [129]:
(bar_data.reset_index().iloc[:,0]*bar_data.reset_index().iloc[:,1])

0     0.013371
1     2.918171
2     0.042472
3    22.530778
4     0.401123
5     2.015644
6     1.042920
7     2.166065
dtype: float64